In [1]:
import numpy as np
from dmbtools.wellbore import survey as srvy
from dmbtools.geometry import utils as gu

In [2]:
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

In [3]:
def angle(u, v):
    """
    Returns the unoriented angle between two non-singular unit length vectors.
    """
    dims = np.ndim(u)
    is_1d = dims == 1
    u = np.atleast_2d(u)
    v = np.atleast_2d(v)
    ang = 2.0 * np.arctan(np.linalg.norm(v - u, axis=1) / np.linalg.norm(u + v, axis=1))
    return ang[0] if is_1d else ang

In [4]:
def _sin_over_x(x):
        """
        Numerically stable sin_over_x function.
        """
        x = x.copy()
        mask = 1.0 + (x * x) == 1.0
        x[mask] = 1.0
        x[~mask] = np.sin(x[~mask]) / x[~mask]
        return x

In [5]:
def slerp2(t, u, v, theta=None):
    """
    https://www.plunk.org/~hatch/rightway.php
    Spherical linear interpolation.
    Returns a unit vector a fraction, t, between unit vectors u and v.
    t: an array of floats to interpolate.
    u: an array of unit direction vectors at the begining of the arc.
    v: an array of unit direction vectors at the end of the arc.
    theta: the angle between u and v. If none then the angle is calculated.
    """
    def _sin_over_x(x):
        """
        Numerically stable sin_over_x function.
        """
        x = x.copy()
        mask = 1.0 + (x * x) == 1.0
        x[mask] = 1.0
        x[~mask] = np.sin(x[~mask]) / x[~mask]
        return x
    
    if theta is None:
        theta = angle(u, v) #2.0 * np.arctan(np.linalg.norm(v - u) / np.linalg.norm(u + v))
    
    # q = 1.0 - t
    # d = _sin_over_x(np.atleast_1d(theta))
    # l = (_sin_over_x(q * theta) / d) * (q * u)
    # r = (_sin_over_x(t * theta) / d) * (t * v)
    # w = l + r
    
    q = 1.0 - t
    d = _sin_over_x(np.atleast_1d(theta))
    l = ((_sin_over_x(q * theta) / d)[:,None]) * (q[:,None] * u)
    r = ((_sin_over_x(t * theta) / d)[:,None]) * (t[:,None] * v)
    w = l + r
    
    return w

In [6]:
def slerp(t, u, v, theta=None):
    """
    https://www.plunk.org/~hatch/rightway.php
    Spherical linear interpolation.
    Returns a unit vector a fraction, t, between unit vectors u and v.
    t: a float or an array of floats to interpolate.
    u and v: are unit vectors.
    theta: the angle between u and v. If none then the angle is calculated.
    """
    def _sin_over_x(x):
        """
        Numerically stable sin_over_x function.
        """
        mask = 1.0 + (x * x) == 1.0
        x[mask] = 1.0
        x[~mask] = np.sin(x[~mask]) / x[~mask]
        return x
    
    is_scalar = np.ndim(t) == 0
    t = np.atleast_1d(t)[:,None]
    if theta is None:
        theta = 2.0 * np.arctan(np.linalg.norm(v - u) / np.linalg.norm(u + v))
    
    q = 1.0 - t
    d = _sin_over_x(np.atleast_1d(theta))[0]
    l = (_sin_over_x(q * theta) / d) * (q * u)
    r = (_sin_over_x(t * theta) / d) * (t * v)
    w = l + r
    
    return w[0] if is_scalar else w

#### Read in the Compendium One test surveys

In [7]:
st_srv, st_tie_in = srvy.S_and_T_survey()

In [8]:
st_srv

array([[702.550, 5.500, 45.000],
       [1964.570, 29.750, 77.050],
       [5086.350, 29.750, 77.050],
       [9901.680, 120.000, 285.000]])

In [9]:
head, pos = srvy.S_and_T_pos_log()

In [10]:
vs = gu.toUnitDir(st_srv[:,1], st_srv[:,2])
vs

array([[0.068, 0.068, 0.995],
       [0.111, 0.484, 0.868],
       [0.111, 0.484, 0.868],
       [0.224, -0.837, -0.500]])

In [11]:
t = np.arange(0., 1.1, 0.25)
t

array([0.000, 0.250, 0.500, 0.750, 1.000])

In [12]:
tt = t[:,None]
tt

array([[0.000],
       [0.250],
       [0.500],
       [0.750],
       [1.000]])

In [13]:
gu.slerp(t, vs[0], vs[1])

array([[0.068, 0.068, 0.995],
       [0.080, 0.176, 0.981],
       [0.092, 0.283, 0.955],
       [0.102, 0.385, 0.917],
       [0.111, 0.484, 0.868]])

In [14]:
gu.slerp2(t, np.array([vs[0]] * 5), np.array([vs[1]] * 5))

array([[0.068, 0.068, 0.995],
       [0.080, 0.176, 0.981],
       [0.092, 0.283, 0.955],
       [0.102, 0.385, 0.917],
       [0.111, 0.484, 0.868]])

In [15]:
u = vs[0]
v = vs[1]
v

array([0.111, 0.484, 0.868])

In [16]:
uu = np.array([vs[0]] * 5)
vv = np.array([vs[1]] * 5)
vv

array([[0.111, 0.484, 0.868],
       [0.111, 0.484, 0.868],
       [0.111, 0.484, 0.868],
       [0.111, 0.484, 0.868],
       [0.111, 0.484, 0.868]])

In [17]:
theta = 2.0 * np.arctan(np.linalg.norm(v - u) / np.linalg.norm(u + v))
theta

0.44055960263249955

In [18]:
thetax = angle(uu, vv)
thetax

array([0.441, 0.441, 0.441, 0.441, 0.441])

In [19]:
q = 1.0 - tt
q

array([[1.000],
       [0.750],
       [0.500],
       [0.250],
       [0.000]])

In [20]:
qq = 1.0 - t
qq

array([1.000, 0.750, 0.500, 0.250, 0.000])

In [21]:
d = _sin_over_x(np.atleast_1d(theta))[0]
d

0.9679636925658849

In [22]:
dd = _sin_over_x(np.atleast_1d(thetax))
dd

array([0.968, 0.968, 0.968, 0.968, 0.968])

In [23]:
l = (_sin_over_x(q * theta) / d) * (q * u)
l

array([[0.068, 0.068, 0.995],
       [0.052, 0.052, 0.757],
       [0.035, 0.035, 0.510],
       [0.017, 0.017, 0.257],
       [0.000, 0.000, 0.000]])

In [24]:
(q * theta)

array([[0.441],
       [0.330],
       [0.220],
       [0.110],
       [0.000]])

In [25]:
ll = ((_sin_over_x(qq * thetax) / dd)[:,None]) * (qq[:,None] * uu)
ll

array([[0.068, 0.068, 0.995],
       [0.052, 0.052, 0.757],
       [0.035, 0.035, 0.510],
       [0.017, 0.017, 0.257],
       [0.000, 0.000, 0.000]])

In [26]:
(qq * thetax)

array([0.441, 0.330, 0.220, 0.110, 0.000])

In [27]:
rr = ((_sin_over_x(t * thetax) / dd)[:,None]) * (t[:,None] * vv)
rr

array([[0.000, 0.000, 0.000],
       [0.029, 0.125, 0.224],
       [0.057, 0.248, 0.445],
       [0.085, 0.368, 0.661],
       [0.111, 0.484, 0.868]])

In [28]:
ww = ll + rr
ww

array([[0.068, 0.068, 0.995],
       [0.080, 0.176, 0.981],
       [0.092, 0.283, 0.955],
       [0.102, 0.385, 0.917],
       [0.111, 0.484, 0.868]])